In [30]:
import pandas as pd

- `index_col="ID"` -> для столбца с индексом присваивается название `"ID"`

In [31]:
data = pd.read_csv("data/auto.csv", index_col="ID")

data

,CarNumber,Make_n_model,Refund,Fines,History
ID,,,,,
0,Y163O8161RUS,Ford Focus,2.0,3200.0,NaN
1,E432XX77RUS,Toyota Camry,1.0,6500.0,NaN
2,7184TT36RUS,Ford Focus,1.0,2100.0,NaN
3,X582HE161RUS,Ford Focus,2.0,2000.0,NaN
4,E34877152RUS,Ford Focus,2.0,6100.0,NaN
...,...,...,...,...,...
926,Y163O8161RUS,Ford Focus,2.0,1600.0,NaN
927,M0309X197RUS,Ford Focus,1.0,22300.0,NaN
928,O673E8197RUS,Ford Focus,2.0,600.0,NaN


## Количество наблюдений

In [32]:
data.count()

CarNumber       931
Make_n_model    931
Refund          914
Fines           869
History          82
dtype: int64

## Работа с дубликатами
- С столбцами `CarNumber`, `Make_n_model`, `Fines`.

- `data = data.drop_duplicates(subset=['CarNumber', 'Make_n_model', 'Fines'], keep="last")`

    - `drop_duplicates()` -> удаляет дублирующиеся строки в датафрейме *data* на основе столбцов: `'CarNumber'`, `'Make_n_model'`, `'Fines'`

    - `subset=['CarNumber', 'Make_n_model', 'Fines']` -> дубликаты определяются по комбинации этих трёх значений. Если две или более строки имеют одинаковые значения в `'CarNumber'`, `'Make_n_model'` и `'Fines'`

    - `keep="last"` -> при обнаружении дубликатов сохраняется последняя строка, а все предыдущие дублирующиеся строки удаляются

In [33]:
data = data.drop_duplicates(subset=['CarNumber', 'Make_n_model', 'Fines'], keep="last")

data.count()

CarNumber       725
Make_n_model    725
Refund          713
Fines           665
History          65
dtype: int64

## Считаем сколько пропущенных значиний с каждого столбца

- `data.isna()` -> создаёт такой же датафрейм что и *data*, где значение `True` соответствует ячейкам с `NaN`, а `False` не пустые ячейки

- `sum()` -> суммирует значения `True`

In [34]:
data.isna().sum()

CarNumber         0
Make_n_model      0
Refund           12
Fines            60
History         660
dtype: int64

## Удаляем столбцы в которых 500 и больше пропущенных значений

- `data = data.dropna(axis=1, thresh=(len(data) - 500))`

    - `dropna()` -> удаляет столбцы `axis=1` из датафрейма *data*, в которых количество непропущенных значений меньше `thresh`

    - `axis=1` -> показатель что нужно удалаять. Если `exis=0` удаляет строку, `exis=1` удаляет столбец

    - `thresh` -> определяет минимальное количество непропущенных значений

    - `thresh=(len(data) - 500)` -> `len(data)` - количество строк в датафрейме. Столбец должен содержать не менее `len(data) - 500` непропущенных значений, чтобы остаться в датафрейме. Все остальное будет удалено

In [35]:
data = data.dropna(axis=1, thresh=(len(data) - 500))

data.isna().sum()

CarNumber        0
Make_n_model     0
Refund          12
Fines           60
dtype: int64

## Заменяем пропущенные значения на предыдущие с столбца `Refund` 

- `data['Refund'] = data['Refund'].ffill()`

    - `ffill()` -> заполняет пропущенные NaN в столбце `'Refund'` датафрейма *data* значениями из предыдущей непропущенной строки

    - Если в строке есть `NaN` в столбце `'Refund'`, оно заменяется значением из ближайшей предыдущей строки, где значение `'Refund'` не является `NaN`

In [36]:
data['Refund'] = data['Refund'].ffill()

data.isna().sum()

CarNumber        0
Make_n_model     0
Refund           0
Fines           60
dtype: int64

## Заменяем пропущенные значения на среднее значения в столбце `Fines`

- `mean_fines = data['Fines'].mean()`

    - Вычисляет среднее арифметическое значение `'Fines'` датафрейма *data*. `NaN` автоматически игнорируются при вычислении

- `data['Fines'] = data['Fines'].fillna(mean_fines)`

    - `fillna(mean_fines)` -> заменяет все `NaN` в столбце `'Fines'` на вычисленное среднее значение `mean_fines`

    - Изменения применяются к столбцу `'Fines'`

In [37]:
mean_fines = data['Fines'].mean()

data['Fines'] = data['Fines'].fillna(mean_fines)

data.isna().sum()

CarNumber       0
Make_n_model    0
Refund          0
Fines           0
dtype: int64

## Делим столбец `Make_n_model` на 2 стобца - `Make`, `Model`

- `data[['Make', 'Model']] = data['Make_n_model'].apply(lambda x: pd.Series(str(x).split(' ', 1)))`

    - `data['Make_n_model']` -> обращается к столбцу `'Make_n_model'` в датафрейме *data*

    - `str(x).split(' ', 1)` -> преобразует значение `x` в строку и разделяет его на две части по первому пробелу. Параметр `1` в `split` ограничивает разделение только первым пробелом, чтобы модели с пробелами корректно разделялись

    - `pd.Series(str(x).split(' ', 1))` -> преобразует полученный список в объект `pandas Series`. Первый элемент становится значением для столбца `'Make'` второй для `'Model'`

    - `apply(lambda x: pd.Series(str(x).split(' ', 1)))` -> применяет эту функцию разделения к каждому значению в столбце `'Make_n_model'`

    - `data[['Make', 'Model']]` -> создаёт или обновляет `'Make'` и `'Model'` в датафрейме *data*

---

In [38]:
# data[['Make', 'Model']] = data['Make_n_model'].apply(lambda x: pd.Series(str(x).split(' ', 1)))

---

- `data.iterrows()` -> метод pandas используемый для перебора строк *DataFrame*

    - Он возвращает *итератор*, который выдает пары `index`, `row` для каждой строки в *DataFrame*

    - Позволяет пройти по всем строкам *DataFrame* и получить доступ к индексу строки и её данным

- `isinstance(object, type)` -> функция Python, проверяет, является ли `object` экземпляром указанного типа `type` или одного из типов, если передан кортеж типов

    - Используется для проверки типа данных, чтобы избежать ошибок при работе с некорректными типами

In [ ]:
data['Make'] = ''
data['Model'] = ''

def split_make_model(value):
    if isinstance(value, str) and value.strip():
        parts = value.split(' ', 1)
        if len(parts) == 1:
            return parts[0], ''
        return parts[0], parts[1]
    else:
        return '', ''

for index, row in data.iterrows():
    make, model = split_make_model(row['Make_n_model'])
    data.at[index, 'Make'] = make
    data.at[index, 'Model'] = model

## Удаляем столбец `Make_n_model`

In [41]:
data = data.drop(columns=["Make_n_model"])

data

,CarNumber,Refund,Fines,Make,Model
ID,,,,,
0,Y163O8161RUS,2.0,3200.000000,Ford,Focus
1,E432XX77RUS,1.0,6500.000000,Toyota,Camry
2,7184TT36RUS,1.0,2100.000000,Ford,Focus
3,X582HE161RUS,2.0,2000.000000,Ford,Focus
5,92918M178RUS,1.0,5700.000000,Ford,Focus
...,...,...,...,...,...
926,Y163O8161RUS,2.0,1600.000000,Ford,Focus
927,M0309X197RUS,1.0,22300.000000,Ford,Focus
928,O673E8197RUS,2.0,600.000000,Ford,Focus


In [42]:
data['Make']

ID
0        Ford
1      Toyota
2        Ford
3        Ford
5        Ford
        ...  
926      Ford
927      Ford
928      Ford
929      Ford
930    Toyota
Name: Make, Length: 725, dtype: object

In [43]:
data["Model"]

ID
0        Focus
1        Camry
2        Focus
3        Focus
5        Focus
        ...   
926      Focus
927      Focus
928      Focus
929      Focus
930    Corolla
Name: Model, Length: 725, dtype: object

## Сохроняем все это в `Json`

- `data.to_json("auto.json", orient="records", indent=5)`

    - `to_json()` -> экспортирует датафрейм *data* в файл JSON с именем `"auto.json"`. Kоторый будет создан или перезаписан

    - `orient="records"` -> Данные будут записаны в формате списка записей. Каждая строка датафрейма представлена как отдельный объект `JSON`. Каждый объект содержит пары `"ключ-значение"`

        - `Ключ` -> имена столбцов

        - `Значение` -> данные из соответствующей строки

    - `indent=5` -> задаёт отступы в `JSON` файле - 5 пробелов

In [44]:
data.to_json("auto.json", orient="records", indent=5)